# Carga de librerias

In [1]:
from pymongo import MongoClient
import time
from datetime import datetime, timedelta

client = MongoClient('localhost', 27017)
db = client["test_mongo"]

# Consultas a la base de datos con 1 millón de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 1 millón de documentos ya indexados

In [2]:
coll = db["extracto_cuenta_1m2"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'agencia_1': {'v': 2, 'key': [('agencia', 1)]},
 'monto_1': {'v': 2, 'key': [('monto', 1)]},
 'saldo_1': {'v': 2, 'key': [('saldo', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_1': {'v': 2, 'key': [('nota', 1)]}}

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la colección

In [267]:
start_time = time.time()
results = coll.find({})
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636db33ddfb1bbbb1b7d41fa'), 'fecha': '1956-04-20 13:40:08', 'agencia': 'Cochabamba', 'monto': 28791.8, 'descripcion': 'Pago de cheque compensado', 'saldo': 3922743.44, 'nota': 'aspernatur soluta earum totam consectetur culpa atque nobis dolor assumenda quaerat'}

{'_id': ObjectId('636db33ddfb1bbbb1b7d41fb'), 'fecha': '1995-02-16 01:20:10', 'agencia': 'Chuquisaca', 'monto': 298799.98, 'descripcion': 'Ingreso en efectivo', 'saldo': 411060.2, 'nota': 'facilis assumenda voluptate illo exercitationem repudiandae at officiis laboriosam quia aliquid adipisci'}

{'_id': ObjectId('636db33ddfb1bbbb1b7d41fc'), 'fecha': '1956-09-04 21:08:47', 'agencia': 'Beni', 'monto': 92841.99, 'descripcion': 'Pago de nómina', 'saldo': 3349136.75, 'nota': 'iusto commodi ducimus aliquam'}

{'_id': ObjectId('636db33ddfb1bbbb1b7d41fd'), 'fecha': '2003-11-11 15:10:18', 'agencia': 'Santa Cruz', 'monto': 79639.42, 'descripcion': 'Cobro seguro', 'saldo': 6523331.21, 'nota': 'aperiam a architecto elige

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [44]:
def busqueda_fecha_agencia(start_date, end_date, agencia):
    from_date_str = start_date + "T00:00:00+00:00"
    to_date_str = end_date + "T23:59:59+00:00"
    from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
    to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
    

    search_criteria = {
        
        "$and":
        [
            {
                "fecha":
                {
                    "$gte": str(from_date),
                    "$lte": str(to_date)
                }
            },
            {
                "agencia": agencia
            }
        ]
    }

    return coll.find(search_criteria)

In [269]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636db342dfb1bbbb1b8bd379'), 'fecha': '2019-01-31 03:44:08', 'agencia': 'Cochabamba', 'monto': 37969.85, 'descripcion': 'Pago de cheque compensado', 'saldo': 4372410.9, 'nota': 'aspernatur unde'}

{'_id': ObjectId('636db33edfb1bbbb1b7dad5e'), 'fecha': '2019-01-31 04:23:01', 'agencia': 'Cochabamba', 'monto': 61430.41, 'descripcion': 'Consignación en efectivo', 'saldo': 4018354.02, 'nota': 'voluptate suscipit aliquid repellendus corrupti exercitationem iste'}

{'_id': ObjectId('636db33fdfb1bbbb1b820cb8'), 'fecha': '2019-01-31 10:32:24', 'agencia': 'Cochabamba', 'monto': 277161.29, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 2900425.22, 'nota': 'totam non sunt'}

{'_id': ObjectId('636db342dfb1bbbb1b89476e'), 'fecha': '2019-01-31 11:19:20', 'agencia': 'Cochabamba', 'monto': 80681.37, 'descripcion': 'Pago recibo de agua', 'saldo': 1233506.96, 'nota': 'ex'}

{'_id': ObjectId('636db343dfb1bbbb1b8c338b'), 'fecha': '2019-01-31 12:22:15', 'agencia': 'Cochabamba

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [45]:
def busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion):
    criteria = {
        "$and": [
            {"$or":
                [
                    {"agencia": agencia1},
                    {"agencia": agencia2}
                ]
                    
            },
            {"saldo":
                {
                    "$gte": saldo_min,
                    "$lte": saldo_max
                }
            },
            {"descripcion":
                    descripcion
            }
        ]}

    return coll.find(criteria)

In [271]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636db33ddfb1bbbb1b7d41fe'), 'fecha': '1983-06-07 17:48:33', 'agencia': 'Santa Cruz', 'monto': 310835.49, 'descripcion': 'Retiro en efectivo', 'saldo': 8995130.49, 'nota': 'commodi saepe fugiat debitis veniam hic dolores nesciunt'}

{'_id': ObjectId('636db33ddfb1bbbb1b7d425f'), 'fecha': '1950-11-02 15:53:33', 'agencia': 'La Paz', 'monto': 74017.15, 'descripcion': 'Retiro en efectivo', 'saldo': 9533088.85, 'nota': 'laborum libero voluptatem eligendi corrupti quod labore atque laboriosam suscipit'}

{'_id': ObjectId('636db33ddfb1bbbb1b7d430f'), 'fecha': '1995-12-22 19:08:33', 'agencia': 'La Paz', 'monto': 298367.78, 'descripcion': 'Retiro en efectivo', 'saldo': 4408981.8, 'nota': 'at odit expedita laudantium velit praesentium'}

{'_id': ObjectId('636db33ddfb1bbbb1b7d439f'), 'fecha': '1992-10-14 18:10:50', 'agencia': 'Santa Cruz', 'monto': 109711.13, 'descripcion': 'Retiro en efectivo', 'saldo': 5800061.73, 'nota': 'aspernatur'}

{'_id': ObjectId('636db33ddfb1bbbb1b7d44a6

## Agregación 1: Suma de saldos de una agencia

Se calculará la suma de los saldos de los datos agrupados de una agencia en específico.

In [46]:
def suma_saldo_agencia(agencia):

    criteria = [
        {
        "$group" : 
            {
                "_id":
                {
                    "agencia": agencia
                },  
                "suma":
                {
                    "$sum" : "$saldo"
                }
            }
        }
     ]

    return coll.aggregate(criteria)

In [273]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': {'agencia': 'Beni'}, 'suma': 5005602059074.02}

 Consulta realizada en un tiempo de 0:00:03.087911


## Agregación 2: Promedio de montos por descripción

In [47]:
def promedio_montos_agencia():

    aggregation = [
        {
            "$group" : 
                {
                    "_id": "$agencia", 
                    "Promedio de montos":
                    {
                        "$avg" : "$saldo"
                    }
                }
        },
        {
            "$sort" : {"Suma de montos": -1}
        },
        {
            "$limit": 5
        }
     ]

    return coll.aggregate(aggregation)

In [7]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': 'Oruro', 'Promedio de montos': 5005883.093573329}

{'_id': 'Santa Cruz', 'Promedio de montos': 4996247.226858266}

{'_id': 'Cochabamba', 'Promedio de montos': 5016002.851214867}

{'_id': 'La Paz', 'Promedio de montos': 4997975.865542692}

{'_id': 'Tarija', 'Promedio de montos': 4999937.056731841}

 Consulta realizada en un tiempo de 0:00:01.148753


| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo consulta 1           | 0.1731 ms      |
| Tiempo consulta 2           | 0.3712 ms      |
| Tiempo consulta 3           | 0.1886 ms      |
| Tiempo agregación 1         | 3.0879 s       |
| Tiempo agregación 2         | 1.1487 s       |

---------------------

# Consultas a la base de datos con 10 millones de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 10 millones de documentos indexados

In [3]:
coll = db["extracto_cuenta_10m2"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'agencia_1': {'v': 2, 'key': [('agencia', 1)]},
 'monto_1': {'v': 2, 'key': [('monto', 1)]},
 'saldo_1': {'v': 2, 'key': [('saldo', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_1': {'v': 2, 'key': [('nota', 1)]}}

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la collección

In [277]:
start_time = time.time()
results = coll.find({})
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636dafcedfb1bba8ecc7b225'), 'fecha': '1956-04-20 13:40:08', 'agencia': 'Cochabamba', 'monto': 28791.8, 'descripcion': 'Pago de cheque compensado', 'saldo': 3922743.44, 'nota': 'aspernatur soluta earum totam consectetur culpa atque nobis dolor assumenda quaerat'}

{'_id': ObjectId('636dafcedfb1bba8ecc7b226'), 'fecha': '1995-02-16 01:20:10', 'agencia': 'Chuquisaca', 'monto': 298799.98, 'descripcion': 'Ingreso en efectivo', 'saldo': 411060.2, 'nota': 'facilis assumenda voluptate illo exercitationem repudiandae at officiis laboriosam quia aliquid adipisci'}

{'_id': ObjectId('636dafcedfb1bba8ecc7b227'), 'fecha': '1956-09-04 21:08:47', 'agencia': 'Beni', 'monto': 92841.99, 'descripcion': 'Pago de nómina', 'saldo': 3349136.75, 'nota': 'iusto commodi ducimus aliquam'}

{'_id': ObjectId('636dafcedfb1bba8ecc7b228'), 'fecha': '2003-11-11 15:10:18', 'agencia': 'Santa Cruz', 'monto': 79639.42, 'descripcion': 'Cobro seguro', 'saldo': 6523331.21, 'nota': 'aperiam a architecto elige

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [278]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636db05ddfb1bba8ec2a31e5'), 'fecha': '2019-01-31 00:18:50', 'agencia': 'Cochabamba', 'monto': 77775.17, 'descripcion': 'Pago recibo de gas', 'saldo': 9791669.21, 'nota': 'corporis quos omnis aut cumque est natus culpa libero recusandae accusantium'}

{'_id': ObjectId('636daffcdfb1bba8ece9603e'), 'fecha': '2019-01-31 00:48:17', 'agencia': 'Cochabamba', 'monto': 310299.88, 'descripcion': 'Reintegro cajero automático', 'saldo': 597145.53, 'nota': 'dolor animi error quia officia'}

{'_id': ObjectId('636db092dfb1bba8ec4d79fe'), 'fecha': '2019-01-31 00:54:46', 'agencia': 'Cochabamba', 'monto': 125951.39, 'descripcion': 'Compra en comercio', 'saldo': 3481199.55, 'nota': 'odit corporis adipisci dolorem sapiente facilis sit sequi dicta ullam provident deleniti'}

{'_id': ObjectId('636db02edfb1bba8ec0d787b'), 'fecha': '2019-01-31 01:05:48', 'agencia': 'Cochabamba', 'monto': 51541.02, 'descripcion': 'Capitalización de intereses premio', 'saldo': 5292368.52, 'nota': 'ullam id pro

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [279]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636dafcedfb1bba8ecc7b229'), 'fecha': '1983-06-07 17:48:33', 'agencia': 'Santa Cruz', 'monto': 310835.49, 'descripcion': 'Retiro en efectivo', 'saldo': 8995130.49, 'nota': 'commodi saepe fugiat debitis veniam hic dolores nesciunt'}

{'_id': ObjectId('636dafcedfb1bba8ecc7b28a'), 'fecha': '1950-11-02 15:53:33', 'agencia': 'La Paz', 'monto': 74017.15, 'descripcion': 'Retiro en efectivo', 'saldo': 9533088.85, 'nota': 'laborum libero voluptatem eligendi corrupti quod labore atque laboriosam suscipit'}

{'_id': ObjectId('636dafcedfb1bba8ecc7b33a'), 'fecha': '1995-12-22 19:08:33', 'agencia': 'La Paz', 'monto': 298367.78, 'descripcion': 'Retiro en efectivo', 'saldo': 4408981.8, 'nota': 'at odit expedita laudantium velit praesentium'}

{'_id': ObjectId('636dafcedfb1bba8ecc7b3ca'), 'fecha': '1992-10-14 18:10:50', 'agencia': 'Santa Cruz', 'monto': 109711.13, 'descripcion': 'Retiro en efectivo', 'saldo': 5800061.73, 'nota': 'aspernatur'}

{'_id': ObjectId('636dafcedfb1bba8ecc7b4d1

## Agregación 1: Suma de saldos de una agencia

In [282]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': {'agencia': 'Beni'}, 'suma': 49995171479223.2}

 Consulta realizada en un tiempo de 0:00:11.997082


## Agregación 2: Promedio de montos por agencia

In [16]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': 'Tarija', 'Promedio de montos': 4996711.953560543}

{'_id': 'Beni', 'Promedio de montos': 5004452.928049985}

{'_id': 'Santa Cruz', 'Promedio de montos': 4996610.398056968}

{'_id': 'Oruro', 'Promedio de montos': 4997994.325706912}

{'_id': 'Pando', 'Promedio de montos': 4995192.7123481585}

 Consulta realizada en un tiempo de 0:00:29.414090


| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo consulta 1           | 0.1621 ms      |
| Tiempo consulta 2           | 0.3347 ms      |
| Tiempo consulta 3           | 0.1764 ms      |
| Tiempo agregación 1         | 11.9971 s      |
| Tiempo agregación 2         | 29.4141 s      |

-----------------

# Consultas a la base de datos con 30 millones de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 30 millones de documentos indexados

In [4]:
coll = db["extracto_cuenta_30m"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'agencia_1': {'v': 2, 'key': [('agencia', 1)]},
 'monto_1': {'v': 2, 'key': [('monto', 1)]},
 'saldo_1': {'v': 2, 'key': [('saldo', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_1': {'v': 2, 'key': [('nota', 1)]}}

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la collección

In [285]:
start_time = time.time()
results = coll.find({})
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d01d'), 'fecha': '1956-04-20 13:40:08', 'agencia': 'Cochabamba', 'monto': 28791.8, 'descripcion': 'Pago de cheque compensado', 'saldo': 3922743.44, 'nota': 'aspernatur soluta earum totam consectetur culpa atque nobis dolor assumenda quaerat'}

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d01e'), 'fecha': '1995-02-16 01:20:10', 'agencia': 'Chuquisaca', 'monto': 298799.98, 'descripcion': 'Ingreso en efectivo', 'saldo': 411060.2, 'nota': 'facilis assumenda voluptate illo exercitationem repudiandae at officiis laboriosam quia aliquid adipisci'}

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d01f'), 'fecha': '1956-09-04 21:08:47', 'agencia': 'Beni', 'monto': 92841.99, 'descripcion': 'Pago de nómina', 'saldo': 3349136.75, 'nota': 'iusto commodi ducimus aliquam'}

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d020'), 'fecha': '2003-11-11 15:10:18', 'agencia': 'Santa Cruz', 'monto': 79639.42, 'descripcion': 'Cobro seguro', 'saldo': 6523331.21, 'nota': 'aperiam a architecto elige

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [286]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636d3fa0dfb1bb4dd61c4fdd'), 'fecha': '2019-01-31 00:18:50', 'agencia': 'Cochabamba', 'monto': 77775.17, 'descripcion': 'Pago recibo de gas', 'saldo': 9791669.21, 'nota': 'corporis quos omnis aut cumque est natus culpa libero recusandae accusantium'}

{'_id': ObjectId('636d4139dfb1bb4dd62a1831'), 'fecha': '2019-01-31 00:22:35', 'agencia': 'Cochabamba', 'monto': 83669.19, 'descripcion': 'Pago recibo de agua', 'saldo': 1739565.27, 'nota': 'aperiam minus ducimus excepturi earum impedit deleniti laudantium sint id doloribus cum'}

{'_id': ObjectId('636d3f49dfb1bb4dd6db7e36'), 'fecha': '2019-01-31 00:48:17', 'agencia': 'Cochabamba', 'monto': 310299.88, 'descripcion': 'Reintegro cajero automático', 'saldo': 597145.53, 'nota': 'dolor animi error quia officia'}

{'_id': ObjectId('636d3fd1dfb1bb4dd63f97f6'), 'fecha': '2019-01-31 00:54:46', 'agencia': 'Cochabamba', 'monto': 125951.39, 'descripcion': 'Compra en comercio', 'saldo': 3481199.55, 'nota': 'odit corporis adipisci dolor

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [287]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d021'), 'fecha': '1983-06-07 17:48:33', 'agencia': 'Santa Cruz', 'monto': 310835.49, 'descripcion': 'Retiro en efectivo', 'saldo': 8995130.49, 'nota': 'commodi saepe fugiat debitis veniam hic dolores nesciunt'}

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d082'), 'fecha': '1950-11-02 15:53:33', 'agencia': 'La Paz', 'monto': 74017.15, 'descripcion': 'Retiro en efectivo', 'saldo': 9533088.85, 'nota': 'laborum libero voluptatem eligendi corrupti quod labore atque laboriosam suscipit'}

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d132'), 'fecha': '1995-12-22 19:08:33', 'agencia': 'La Paz', 'monto': 298367.78, 'descripcion': 'Retiro en efectivo', 'saldo': 4408981.8, 'nota': 'at odit expedita laudantium velit praesentium'}

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d1c2'), 'fecha': '1992-10-14 18:10:50', 'agencia': 'Santa Cruz', 'monto': 109711.13, 'descripcion': 'Retiro en efectivo', 'saldo': 5800061.73, 'nota': 'aspernatur'}

{'_id': ObjectId('636d3f10dfb1bb4dd6b9d2c9

## Agregación 1: Suma de saldos de una agencia

In [288]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': {'agencia': 'Beni'}, 'suma': 149979123171265.12}

 Consulta realizada en un tiempo de 0:11:00.092000


## Agregación 2: Promedio de montos por agencia

In [18]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': 'Santa Cruz', 'Promedio de montos': 4998317.884811896}

{'_id': 'Beni', 'Promedio de montos': 5000906.687422729}

{'_id': 'Tarija', 'Promedio de montos': 4996939.1596396165}

{'_id': 'Cochabamba', 'Promedio de montos': 4999107.784366417}

{'_id': 'Oruro', 'Promedio de montos': 4999753.750790663}

 Consulta realizada en un tiempo de 0:46:31.060006


| Tabla resumen de resultados | Valor obtenido   |
|-----------------------------|------------------|
| Tiempo consulta 1           | 0.1769 ms        |
| Tiempo consulta 2           | 1.7519 ms        |
| Tiempo consulta 3           | 0.2115 ms        |
| Tiempo agregación 1         | 11 min 0.0920 s  |
| Tiempo agregación 2         | 46 min 31.0600 s |

----------------

# Consultas a la base de datos con 10 millones de registros y 3 campos indexados (fecha, descripcion, nota)

En esta sección se revisará la operación de búsqueda para la colección de 10 millones de documentos indexados solamente en 3 campos: fecha, descripcion y nota.

In [65]:
coll = db["extracto_cuenta_10m_3idx"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('nota', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la collección

In [42]:
start_time = time.time()
results = coll.find({})
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('63740c18dfb1bb2837b052b3'), 'fecha': '1956-04-20 13:40:08', 'agencia': 'Cochabamba', 'monto': 28791.8, 'descripcion': 'Pago de cheque compensado', 'saldo': 3922743.44, 'nota': 'aspernatur soluta earum totam consectetur culpa atque nobis dolor assumenda quaerat'}

{'_id': ObjectId('63740c18dfb1bb2837b052b4'), 'fecha': '1995-02-16 01:20:10', 'agencia': 'Chuquisaca', 'monto': 298799.98, 'descripcion': 'Ingreso en efectivo', 'saldo': 411060.2, 'nota': 'facilis assumenda voluptate illo exercitationem repudiandae at officiis laboriosam quia aliquid adipisci'}

{'_id': ObjectId('63740c18dfb1bb2837b052b5'), 'fecha': '1956-09-04 21:08:47', 'agencia': 'Beni', 'monto': 92841.99, 'descripcion': 'Pago de nómina', 'saldo': 3349136.75, 'nota': 'iusto commodi ducimus aliquam'}

{'_id': ObjectId('63740c18dfb1bb2837b052b6'), 'fecha': '2003-11-11 15:10:18', 'agencia': 'Santa Cruz', 'monto': 79639.42, 'descripcion': 'Cobro seguro', 'saldo': 6523331.21, 'nota': 'aperiam a architecto elige

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [49]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('63740ca2dfb1bb283712d273'), 'fecha': '2019-01-31 00:18:50', 'agencia': 'Cochabamba', 'monto': 77775.17, 'descripcion': 'Pago recibo de gas', 'saldo': 9791669.21, 'nota': 'corporis quos omnis aut cumque est natus culpa libero recusandae accusantium'}

{'_id': ObjectId('63740c4cdfb1bb2837d200cc'), 'fecha': '2019-01-31 00:48:17', 'agencia': 'Cochabamba', 'monto': 310299.88, 'descripcion': 'Reintegro cajero automático', 'saldo': 597145.53, 'nota': 'dolor animi error quia officia'}

{'_id': ObjectId('63740ccfdfb1bb2837361a8c'), 'fecha': '2019-01-31 00:54:46', 'agencia': 'Cochabamba', 'monto': 125951.39, 'descripcion': 'Compra en comercio', 'saldo': 3481199.55, 'nota': 'odit corporis adipisci dolorem sapiente facilis sit sequi dicta ullam provident deleniti'}

{'_id': ObjectId('63740c79dfb1bb2837f61908'), 'fecha': '2019-01-31 01:05:48', 'agencia': 'Cochabamba', 'monto': 51541.02, 'descripcion': 'Capitalización de intereses premio', 'saldo': 5292368.52, 'nota': 'ullam id pro

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [50]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('63740c18dfb1bb2837b052b7'), 'fecha': '1983-06-07 17:48:33', 'agencia': 'Santa Cruz', 'monto': 310835.49, 'descripcion': 'Retiro en efectivo', 'saldo': 8995130.49, 'nota': 'commodi saepe fugiat debitis veniam hic dolores nesciunt'}

{'_id': ObjectId('63740c18dfb1bb2837b05318'), 'fecha': '1950-11-02 15:53:33', 'agencia': 'La Paz', 'monto': 74017.15, 'descripcion': 'Retiro en efectivo', 'saldo': 9533088.85, 'nota': 'laborum libero voluptatem eligendi corrupti quod labore atque laboriosam suscipit'}

{'_id': ObjectId('63740c18dfb1bb2837b053c8'), 'fecha': '1995-12-22 19:08:33', 'agencia': 'La Paz', 'monto': 298367.78, 'descripcion': 'Retiro en efectivo', 'saldo': 4408981.8, 'nota': 'at odit expedita laudantium velit praesentium'}

{'_id': ObjectId('63740c18dfb1bb2837b05458'), 'fecha': '1992-10-14 18:10:50', 'agencia': 'Santa Cruz', 'monto': 109711.13, 'descripcion': 'Retiro en efectivo', 'saldo': 5800061.73, 'nota': 'aspernatur'}

{'_id': ObjectId('63740c18dfb1bb2837b0555f

## Consulta 4: Fecha, descripción y nota

Se realizará una búsqueda sobre los campos fecha, descripcion y nota. Los parámetros de entrada son un rango de fechas, una descripción, y tres palabras del lorem ipsum.

In [57]:
def fecha_descripcion_nota(start_date, end_date, descripcion, words):    
    from_date_str = start_date + "T00:00:00+00:00"
    to_date_str = end_date + "T23:59:59+00:00"
    from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
    to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')

    search_criteria = {
        
        "$and":
        [
            {
                "fecha":
                {
                    "$gte": str(from_date),
                    "$lte": str(to_date)
                }
            },
            {
                "descripcion": descripcion
            },
            {
                "$text": {"$search": words}
            }
        ]
    }

    return coll.find(search_criteria)

In [66]:
words = 'lorem ipsum omnis'
descripcion = "Retiro en efectivo"
start_date = "2019-01-31" 
end_date = "2019-12-31"

start_time = time.time()
results = fecha_descripcion_nota(start_date, end_date, descripcion, words)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('63740cb7dfb1bb28371fe5d3'), 'fecha': '2019-12-05 14:37:46', 'agencia': 'Beni', 'monto': 186710.76, 'descripcion': 'Retiro en efectivo', 'saldo': 4128920.84, 'nota': 'ipsum'}

{'_id': ObjectId('63740cbadfb1bb28372733ea'), 'fecha': '2019-08-18 10:17:08', 'agencia': 'Pando', 'monto': 120456.19, 'descripcion': 'Retiro en efectivo', 'saldo': 4622608.35, 'nota': 'mollitia ipsum'}

{'_id': ObjectId('63740ca1dfb1bb28370f1ba6'), 'fecha': '2019-12-22 15:01:01', 'agencia': 'Beni', 'monto': 81562.84, 'descripcion': 'Retiro en efectivo', 'saldo': 1342224.84, 'nota': 'ipsum neque'}

{'_id': ObjectId('63740c7adfb1bb2837f8709d'), 'fecha': '2019-05-11 07:50:17', 'agencia': 'Santa Cruz', 'monto': 257114.43, 'descripcion': 'Retiro en efectivo', 'saldo': 2480857.91, 'nota': 'ipsum magni'}

{'_id': ObjectId('63740c65dfb1bb2837e9a6c8'), 'fecha': '2019-12-22 15:13:39', 'agencia': 'Pando', 'monto': 222750.08, 'descripcion': 'Retiro en efectivo', 'saldo': 503592.65, 'nota': 'architecto ipsum'

## Agregación 1: Suma de saldos de una agencia

In [52]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': {'agencia': 'Beni'}, 'suma': 49995171479223.2}

 Consulta realizada en un tiempo de 0:01:56.482251


## Agregación 2: Promedio de montos por agencia

In [53]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': 'Tarija', 'Promedio de montos': 4996711.953560543}

{'_id': 'Cochabamba', 'Promedio de montos': 5000650.992326564}

{'_id': 'Beni', 'Promedio de montos': 5004452.928049985}

{'_id': 'Chuquisaca', 'Promedio de montos': 5004312.36702801}

{'_id': 'Oruro', 'Promedio de montos': 4997994.325706912}

 Consulta realizada en un tiempo de 0:00:14.817012


| Tabla resumen de resultados | Valor obtenido  |
|-----------------------------|-----------------|
| Tiempo consulta 1           | 0.1740 ms       |
| Tiempo consulta 2           | 0.3581 ms       |
| Tiempo consulta 3           | 0.1850 ms       |
| Tiempo consulta 4           | 0.2942 ms       |
| Tiempo agregación 1         | 1 min 56.4822 s |
| Tiempo agregación 2         | 14.8170 s       |

----------------------------

# Consultas a la base de datos con 30 millones de registros y 3 campos indexados (fecha, descripcion, nota)

En esta sección se revisará la operación de búsqueda para la colección de 30 millones de documentos indexados solamente en 3 campos: fecha, descripcion y nota.

In [67]:
coll = db["extracto_cuenta_30m_3idx"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('nota', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

## Consulta 1: Todos los datos

Se realizará una búsqueda sobre todos los campos en la collección

In [68]:
start_time = time.time()
results = coll.find({})
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('63740eecdfb1bb3887b98884'), 'fecha': '1956-04-20 13:40:08', 'agencia': 'Cochabamba', 'monto': 28791.8, 'descripcion': 'Pago de cheque compensado', 'saldo': 3922743.44, 'nota': 'aspernatur soluta earum totam consectetur culpa atque nobis dolor assumenda quaerat'}

{'_id': ObjectId('63740eecdfb1bb3887b98885'), 'fecha': '1995-02-16 01:20:10', 'agencia': 'Chuquisaca', 'monto': 298799.98, 'descripcion': 'Ingreso en efectivo', 'saldo': 411060.2, 'nota': 'facilis assumenda voluptate illo exercitationem repudiandae at officiis laboriosam quia aliquid adipisci'}

{'_id': ObjectId('63740eecdfb1bb3887b98886'), 'fecha': '1956-09-04 21:08:47', 'agencia': 'Beni', 'monto': 92841.99, 'descripcion': 'Pago de nómina', 'saldo': 3349136.75, 'nota': 'iusto commodi ducimus aliquam'}

{'_id': ObjectId('63740eecdfb1bb3887b98887'), 'fecha': '2003-11-11 15:10:18', 'agencia': 'Santa Cruz', 'monto': 79639.42, 'descripcion': 'Cobro seguro', 'saldo': 6523331.21, 'nota': 'aperiam a architecto elige

## Consulta 2: Fecha y agencia

Se realizará una búsqueda sobre los campos fecha y agencia. Siendo los parámetros de entrada una fecha inicial, una fecha final y una agencia.

In [69]:
start_date = "2019-01-31" 
end_date = "2019-12-31"
agencia = "Cochabamba"

start_time = time.time()
results = busqueda_fecha_agencia(start_date, end_date, agencia, coll)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('63740f7fdfb1bb38871c0844'), 'fecha': '2019-01-31 00:18:50', 'agencia': 'Cochabamba', 'monto': 77775.17, 'descripcion': 'Pago recibo de gas', 'saldo': 9791669.21, 'nota': 'corporis quos omnis aut cumque est natus culpa libero recusandae accusantium'}

{'_id': ObjectId('637410f2dfb1bb388729d098'), 'fecha': '2019-01-31 00:22:35', 'agencia': 'Cochabamba', 'monto': 83669.19, 'descripcion': 'Pago recibo de agua', 'saldo': 1739565.27, 'nota': 'aperiam minus ducimus excepturi earum impedit deleniti laudantium sint id doloribus cum'}

{'_id': ObjectId('63740f24dfb1bb3887db369d'), 'fecha': '2019-01-31 00:48:17', 'agencia': 'Cochabamba', 'monto': 310299.88, 'descripcion': 'Reintegro cajero automático', 'saldo': 597145.53, 'nota': 'dolor animi error quia officia'}

{'_id': ObjectId('63740fa9dfb1bb38873f505d'), 'fecha': '2019-01-31 00:54:46', 'agencia': 'Cochabamba', 'monto': 125951.39, 'descripcion': 'Compra en comercio', 'saldo': 3481199.55, 'nota': 'odit corporis adipisci dolor

## Consulta 3: Rango de saldo, dos opciones de agencias y descripción

Se realizará una búsqueda sobre los campos saldo, agencia (OR 2 opciones) y descripción. Los parámetros de entrada son un rango de saldos, dos opciones de agencias, y una descripción.

In [70]:
saldo_min = 162342
saldo_max = 382304513
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = "Retiro en efectivo"

start_time = time.time()
results = busqueda_saldo_agencias_descripcion(saldo_min, saldo_max, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('63740eecdfb1bb3887b98888'), 'fecha': '1983-06-07 17:48:33', 'agencia': 'Santa Cruz', 'monto': 310835.49, 'descripcion': 'Retiro en efectivo', 'saldo': 8995130.49, 'nota': 'commodi saepe fugiat debitis veniam hic dolores nesciunt'}

{'_id': ObjectId('63740eecdfb1bb3887b988e9'), 'fecha': '1950-11-02 15:53:33', 'agencia': 'La Paz', 'monto': 74017.15, 'descripcion': 'Retiro en efectivo', 'saldo': 9533088.85, 'nota': 'laborum libero voluptatem eligendi corrupti quod labore atque laboriosam suscipit'}

{'_id': ObjectId('63740eecdfb1bb3887b98999'), 'fecha': '1995-12-22 19:08:33', 'agencia': 'La Paz', 'monto': 298367.78, 'descripcion': 'Retiro en efectivo', 'saldo': 4408981.8, 'nota': 'at odit expedita laudantium velit praesentium'}

{'_id': ObjectId('63740eecdfb1bb3887b98a29'), 'fecha': '1992-10-14 18:10:50', 'agencia': 'Santa Cruz', 'monto': 109711.13, 'descripcion': 'Retiro en efectivo', 'saldo': 5800061.73, 'nota': 'aspernatur'}

{'_id': ObjectId('63740eecdfb1bb3887b98b30

## Consulta 4: Fecha, descripción y nota

Se realizará una búsqueda sobre los campos fecha, descripcion y nota. Los parámetros de entrada son un rango de fechas, una descripción, y tres palabras del lorem ipsum.

In [75]:
words = 'lorem ipsum omnis'
descripcion = "Retiro en efectivo"
start_date = "2019-01-31" 
end_date = "2019-12-31"

start_time = time.time()
results = fecha_descripcion_nota(start_date, end_date, descripcion, words)
end_time = time.time()

print("="*100)

for r in results[:10]:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {round((end_time - start_time)*1000, 4)} milisegundos")
print("="*100)

{'_id': ObjectId('63741106dfb1bb38873ba25c'), 'fecha': '2019-06-25 15:31:50', 'agencia': 'Oruro', 'monto': 399452.49, 'descripcion': 'Retiro en efectivo', 'saldo': 9229518.64, 'nota': 'ipsum'}

{'_id': ObjectId('63740f94dfb1bb3887291ba4'), 'fecha': '2019-12-05 14:37:46', 'agencia': 'Beni', 'monto': 186710.76, 'descripcion': 'Retiro en efectivo', 'saldo': 4128920.84, 'nota': 'ipsum'}

{'_id': ObjectId('63741109dfb1bb388743e20e'), 'fecha': '2019-10-12 03:42:02', 'agencia': 'Beni', 'monto': 185366.08, 'descripcion': 'Retiro en efectivo', 'saldo': 4732418.74, 'nota': 'distinctio ipsum'}

{'_id': ObjectId('63741105dfb1bb388739c56a'), 'fecha': '2019-03-08 20:17:11', 'agencia': 'Oruro', 'monto': 295444.29, 'descripcion': 'Retiro en efectivo', 'saldo': 8149006.35, 'nota': 'ipsum voluptatibus'}

{'_id': ObjectId('637410e1dfb1bb38871cd05a'), 'fecha': '2019-06-01 20:45:35', 'agencia': 'Cochabamba', 'monto': 275929.16, 'descripcion': 'Retiro en efectivo', 'saldo': 1700276.11, 'nota': 'itaque ipsum

## Agregación 1: Suma de saldos de una agencia

In [76]:
agencia = "Beni"

start_time = time.time()
results = suma_saldo_agencia(agencia)
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': {'agencia': 'Beni'}, 'suma': 149979123171265.12}

 Consulta realizada en un tiempo de 0:06:32.652452


## Agregación 2: Promedio de montos por descripción

In [77]:
start_time = time.time()
results = promedio_montos_agencia()
end_time = time.time()

print("="*100)

for r in results:
    print(r)
    print("")

print("="*100)
print(f" Consulta realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

{'_id': 'Beni', 'Promedio de montos': 5000906.687422729}

{'_id': 'Oruro', 'Promedio de montos': 4999753.750790663}

{'_id': 'Tarija', 'Promedio de montos': 4996939.1596396165}

{'_id': 'Cochabamba', 'Promedio de montos': 4999107.784366417}

{'_id': 'Pando', 'Promedio de montos': 4997057.697427729}

 Consulta realizada en un tiempo de 0:10:11.458729


| Tabla resumen de resultados | Valor obtenido   |
|-----------------------------|------------------|
| Tiempo consulta 1           | 0.1862 ms        |
| Tiempo consulta 2           | 0.2246 ms        |
| Tiempo consulta 3           | 0.1528 ms        |
| Tiempo consulta 4           | 0.2358 ms        |
| Tiempo agregación 1         | 6 min 32.6524  s |
| Tiempo agregación 2         | 10 min 11.4587 s |